In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import tensorflow as tf
import numpy as np
import cv2
import glob

def psnr_metric(y_true, y_pred):
    psnr = tf.image.psnr(y_true, y_pred, max_val=1)
    return tf.reduce_mean(psnr)

def ssim_metric(y_true, y_pred):
    ssim = tf.image.ssim(y_true, y_pred, max_val=1)
    return tf.reduce_mean(ssim)

def msssim_metric(y_true, y_pred):
    msssim = tf.image.ssim_multiscale(y_true, y_pred, max_val=1)
    return tf.reduce_mean(msssim)

In [ ]:
# Settings
filters = 96
blocks = 16
act = "relu"
kernel_size = 3

class DepthToSpace(keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, x):
        x = tf.nn.depth_to_space(x, 2)
        x = keras.ops.clip(x, 0.0, 1.0)
        return x

def res_block(input, filters=filters, kernel_size=kernel_size):
    x = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same', activation=act)(input)
    x = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same', activation=act)(x)
    x = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = keras.layers.Add()([x, input])
    return x

# Build the model:
inputs = keras.layers.Input(shape=(None,None,1))
conv0 = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same')(inputs)

x = res_block(conv0)
for _ in range(blocks - 1):
    x = res_block(x)

# Feature Fusion
conv1 = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same')(x)
mix = keras.layers.Add()([conv1, conv0])

# Upsampler
features = keras.layers.Conv2D(filters=4, kernel_size=kernel_size, padding='same')(mix)
outputs = DepthToSpace()(features)

# Defining the model
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()
# Compile the model.
model.compile(optimizer='Adam', loss='mae', metrics=[psnr_metric, ssim_metric, msssim_metric])

In [ ]:
# Download data from gdrive
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/tmp/r16f96.keras /content/r16f96.keras

!cp /content/drive/MyDrive/Datasets/FHD_Anime.zip /content/FHD_Anime.zip
!unzip /content/FHD_Anime.zip

In [ ]:
model.load_weights("/content/r16f96.keras", skip_mismatch=True)

In [ ]:
# FHD Anime
number_files = len(os.listdir('/content/FHD_Anime/'))
metrics_array = np.empty([number_files, 4])
idx = 0

filelist = sorted(glob.glob('/content/FHD_Anime/*.png'))

for myFile in filelist:
    test_ref = cv2.imread(myFile, cv2.IMREAD_COLOR)
    test_ref = cv2.cvtColor(test_ref, cv2.COLOR_BGR2GRAY, 0)
    test_in = cv2.resize(test_ref, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR_EXACT)

    test_ref = np.array(test_ref).astype(np.float32) / 255.0
    test_ref = np.clip(test_ref, 0.0, 1.0)
    test_ref = np.expand_dims(test_ref, axis=-1)
    test_ref = np.expand_dims(test_ref, axis=0)

    test_in = np.array(test_in).astype(np.float32) / 255.0
    test_in = np.clip(test_in, 0.0, 1.0)
    test_in = np.expand_dims(test_in, axis=-1)
    test_in = np.expand_dims(test_in, axis=0)

    metrics_array[idx, :] = np.array(model.evaluate(test_in, test_ref, verbose=1))
    idx = idx + 1

print(filelist)
print(metrics_array)
np.set_printoptions(suppress=True)
print(np.average(metrics_array, axis=0))

In [ ]:
filelist1 = sorted(glob.glob('/content/FHD_Anime/*.png'))
test_in = []
for myFile in filelist1:
    image = cv2.imread(myFile, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0)
    image = cv2.resize(image, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR_EXACT)
    test_in.append(image)
test_in = np.array(test_in).astype(np.float32) / 255.0
test_in = np.clip(test_in, 0.0, 1.0)
test_in = np.expand_dims(test_in, axis=-1)
print(test_in.shape)

filelist2 = sorted(glob.glob('/content/FHD_Anime/*.png'))
test_ref = []
for myFile in filelist2:
    image = cv2.imread(myFile, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0)
    test_ref.append(image)
test_ref = np.array(test_ref).astype(np.float32) / 255.0
test_ref = np.clip(test_ref, 0.0, 1.0)
test_ref = np.expand_dims(test_ref, axis=-1)
print(test_ref.shape)

metrics_array = np.array(model.evaluate(test_in, test_ref, batch_size=1, verbose=1))
print(metrics_array)